# 포탈 01 - 네이버 파이낸스 종목별 과거 시세

<img src="http://i.imgur.com/grQHNKG.jpg">

#### 2017 FinanceData http://fb.com/financedata

# 종목 마스터 만들기
* 거래소 상장회사검색 http://marketdata.krx.co.kr/contents/MKD/04/0406/04060100/MKD04060100.jsp


In [1]:
import requests
import sqlite3
import pandas as pd
import io

def get_krx_stock_master():
    # STEP 01: Generate OTP
    gen_otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'
    gen_otp_data = {
        'name':'fileDown',
        'filetype':'xls',
        'url':'MKD/04/0406/04060100/mkd04060100_01',
        'market_gubun':'ALL', # ''ALL':전체, STK': 코스피
        'isu_cdnm':'전체',
        'sort_type':'A',
        'std_ind_cd':'01',
        'lst_stk_vl':'1',
        'in_lst_stk_vl':'',
        'in_lst_stk_vl2':'',
        'pagePath':'/contents/MKD/04/0406/04060100/MKD04060100.jsp',
    }

    r = requests.post(gen_otp_url, gen_otp_data)
    code = r.content

    # STEP 02: download
    down_url = 'http://file.krx.co.kr/download.jspx'
    down_data = {
        'code': code,
    }

    r = requests.post(down_url, down_data)
    f = io.BytesIO(r.content)
    
    usecols = ['종목코드', '기업명', '업종코드', '업종', '대표전화', '주소']
    df = pd.read_excel(f, converters={'종목코드': str, '업종코드': str}, usecols=usecols)
    df.columns = ['code', 'name', 'sector_code', 'sector', 'telephone', 'address']
    return df

if __name__ == "__main__":
    conn = sqlite3.connect('findata.db')
    df = get_krx_stock_master()
    df.to_sql('stock_master', conn, if_exists='replace')
    conn.close()

In [2]:
# 저장된 데이터 확인

conn = sqlite3.connect('findata.db')
df = pd.read_sql('select * from stock_master', conn)
df.head(10)

,index,code,name,sector_code,sector,telephone,address
0,0,060310,3S,032602,전자부품 제조업,02-896-9474,서울특별시 금천구 시흥대로71길 30-1
1,1,095570,AJ네트웍스,126903,산업용 기계 및 장비 임대업,02-6240-0800,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)"
2,2,068400,AJ렌터카,126901,운송장비 임대업,1544-1600,서울특별시 구로구 서부샛길 822
3,3,006840,AK홀딩스,137105,"회사본부, 지주회사 및 경영컨설팅 서비스업",02-6923-2921,서울특별시 구로구 구로중앙로 152(구로동)
4,4,054620,APS홀딩스,032902,특수 목적용 기계 제조업,031-776-1800,경기도 화성시 동탄면 동탄산단9길 23-12 (동탄면)
5,5,265520,AP시스템,032902,특수 목적용 기계 제조업,031-379-2700,경기도 화성시 동탄면 동탄산단8길 15-5 &nbsp
6,6,211270,AP위성,032604,통신 및 방송 장비 제조업,02-2026-7800,서울특별시 금천구 가산디지털2로 98 IT캐슬 2동 9층 &nbsp
7,7,027410,BGF리테일,074701,종합 소매업,1577-3663,서울특별시 강남구 테헤란로 405
8,8,138930,BNK금융지주,116409,기타 금융업,051-620-3022,부산광역시 남구 문현금융로 30(문현동)
9,9,001460,BYC,031401,봉제의복 제조업,02-840-3175,서울특별시 영등포구 도림천로21길 3


# 네이버 파이낸스 크롤링

* http://finance.naver.com
* http://finance.naver.com/item/sise_day.nhn?code=105560&page=1

# 대표적인 두가지 방법
1. pd.read_html()
1. BeautifulSoup, requests

# DataFrame으로 읽기

In [3]:
import pandas as pd

url = 'http://finance.naver.com/item/sise_day.nhn?code=105560&page=1'
dfs = pd.read_html(url)

In [4]:
len(dfs)

2

In [5]:
df_price = dfs[0]
df_price

,0,1,2,3,4,5,6
0,날짜,종가,전일비,시가,고가,저가,거래량
1,2017.06.22,54200,0,54000,54400,53800,249046
2,2017.06.21,54200,1200,55400,55400,53700,1257868
3,2017.06.20,55400,200,55100,55900,55000,628465
4,2017.06.19,55600,100,55000,55800,55000,570138
5,2017.06.16,55500,600,54900,55500,54400,938141
6,2017.06.15,54900,800,55200,55900,54200,714613
7,2017.06.14,55700,100,55600,56400,55200,1062118
8,2017.06.13,55600,700,56500,56700,55200,562317
9,2017.06.12,56300,400,56500,56900,55800,870522


In [6]:
df_price.dtypes

0    object
1    object
2    object
3    object
4    object
5    object
6    object
dtype: object

In [7]:
df_price.columns = ['date', 'close', 'change', 'open', 'high', 'low', 'volume']
df_price = df_price[1:] 
df_price = df_price.replace('\.', '-', regex=True)
df_price['date'] = pd.to_datetime(df_price['date'])

int_cols = ['close', 'change', 'open', 'high', 'low', 'volume']
df_price[int_cols] = df_price[int_cols].astype('int')
df_price

,date,close,change,open,high,low,volume
1,2017-06-22,54200,0,54000,54400,53800,249046
2,2017-06-21,54200,1200,55400,55400,53700,1257868
3,2017-06-20,55400,200,55100,55900,55000,628465
4,2017-06-19,55600,100,55000,55800,55000,570138
5,2017-06-16,55500,600,54900,55500,54400,938141
6,2017-06-15,54900,800,55200,55900,54200,714613
7,2017-06-14,55700,100,55600,56400,55200,1062118
8,2017-06-13,55600,700,56500,56700,55200,562317
9,2017-06-12,56300,400,56500,56900,55800,870522
10,2017-06-09,55900,200,55200,56200,55000,1167254


In [8]:
df_price.dtypes

date      datetime64[ns]
close              int64
change             int64
open               int64
high               int64
low                int64
volume             int64
dtype: object

# DB 테이블로 저장

In [9]:
import pandas as pd
import sqlite3

# KB금융(105560)

url = 'http://finance.naver.com/item/sise_day.nhn?code=105560&page=1'
dfs = pd.read_html(url)

In [10]:
conn = sqlite3.connect('findata.db')

df_price.to_sql('test_table', conn, index=False, if_exists='replace')
conn.close()

In [11]:
conn = sqlite3.connect('findata.db')

df_price = pd.read_sql('select * from test_table', conn, parse_dates=['date'])
df_price.head()

,date,close,change,open,high,low,volume
0,2017-06-22,54200,0,54000,54400,53800,249046
1,2017-06-21,54200,1200,55400,55400,53700,1257868
2,2017-06-20,55400,200,55100,55900,55000,628465
3,2017-06-19,55600,100,55000,55800,55000,570138
4,2017-06-16,55500,600,54900,55500,54400,938141


In [12]:
df_price.dtypes

date      datetime64[ns]
close              int64
change             int64
open               int64
high               int64
low                int64
volume             int64
dtype: object

In [13]:
conn.close()

# 페이지 네비게이션
http://finance.naver.com/item/sise_day.nhn?code=105560&page=1
1. 마지막 페이지 숫자 알아내기
1. requests, BeautifulSoup 사용

In [14]:
from bs4 import BeautifulSoup
import requests

In [15]:
url = 'http://finance.naver.com/item/sise_day.nhn?code=105560&page=1'
r = requests.get(url)
soup = BeautifulSoup(r.text, "lxml")

In [16]:
td = soup.find('td', attrs={'class':'pgRR'})
td

<td class="pgRR">
<a href="/item/sise_day.nhn?code=105560&amp;page=216">맨뒤
				<img alt="" border="0" height="5" src="http://static.naver.net/n/cmn/bu_pgarRR.gif" width="8"/>
</a>
</td>

In [17]:
td = soup.find('td', attrs={'class':'pgRR'})
npage = int(td.a['href'].split('page=')[1])
npage

216

In [18]:
from bs4 import BeautifulSoup
import requests

def get_last_page_num(code):
    npage = 1
    url = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=1' % (code)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    td = soup.find('td', attrs={'class':'pgRR'})
    if td:
        npage = td.a['href'].split('page=')[1]
    return int(npage)

In [19]:
get_last_page_num('005930') # 삼성전자

530

In [20]:
get_last_page_num('105560') # KB금융

216

# 페이지 전체 네비게이션

In [21]:
url_tmpl = 'http://finance.naver.com/item/sise_day.nhn?code=105560&page=%d'

npages = get_last_page_num('105560')
for p in range(1, npages + 1):
    url = url_tmpl % p


In [22]:
df_price['code'] = '105560'
df_price

,date,close,change,open,high,low,volume,code
0,2017-06-22,54200,0,54000,54400,53800,249046,105560
1,2017-06-21,54200,1200,55400,55400,53700,1257868,105560
2,2017-06-20,55400,200,55100,55900,55000,628465,105560
3,2017-06-19,55600,100,55000,55800,55000,570138,105560
4,2017-06-16,55500,600,54900,55500,54400,938141,105560
5,2017-06-15,54900,800,55200,55900,54200,714613,105560
6,2017-06-14,55700,100,55600,56400,55200,1062118,105560
7,2017-06-13,55600,700,56500,56700,55200,562317,105560
8,2017-06-12,56300,400,56500,56900,55800,870522,105560
9,2017-06-09,55900,200,55200,56200,55000,1167254,105560


# 전종목 과거 가격 데이터저장

In [23]:
# stock_price 테이블 생성
conn = sqlite3.connect('findata.db')

sql = '''
CREATE TABLE "stock_price" (    
  "date" TIMESTAMP,
  "code" TEXT,
  "close" INTEGER,
  "change" INTEGER,
  "open" INTEGER,
  "high" INTEGER, 
  "low" INTEGER,  
  "volume" INTEGER
);
'''
c = conn.cursor()
c.execute(sql)
conn.close()

OperationalError: table "stock_price" already exists

In [ ]:
import sqlite3
conn = sqlite3.connect('findata.db')

code = '105560'

url_tmpl = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=%d'

npages = get_last_page_num(code)

for p in range(npages, 0, -1):
    url = url_tmpl % (code, p)
    dfs = pd.read_html(url)
    df_price = dfs[0]
    df_price.columns = ['date', 'close', 'change', 'open', 'high', 'low', 'volume']
    df_price = df_price[1:] 
    df_price = df_price.replace('\.', '-', regex=True)
    df_price['date'] = pd.to_datetime(df_price['date'])
    int_cols = ['close', 'change', 'open', 'high', 'low', 'volume']
    df_price[int_cols] = df_price[int_cols].astype('int', errors='ignore')
    df_price['code'] = code
    df_price.to_sql('stock_price', conn, if_exists='append', index=False)
    print('%d,' % p, end='')

In [ ]:
df_price[int_cols]

In [ ]:
import sqlite3
conn = sqlite3.connect('findata.db')

def stock_code_price_to_sql(code):
    df_max = pd.read_sql('SELECT * FROM stock_price where code="%s" ORDER BY date DESC LIMIT 1', conn)
    
    url_tmpl = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=%d'
    npages = get_last_page_num(code)
    for p in range(npages, 0, -1):
        url = url_tmpl % (code, p)
        dfs = pd.read_html(url)
        df_price = dfs[0]
        df_price.columns = ['date', 'close', 'change', 'open', 'high', 'low', 'volume']
        df_price = df_price[1:] 
        df_price = df_price.replace('\.', '-', regex=True)
        df_price['date'] = pd.to_datetime(df_price['date'])
        df_price[int_cols] = df_price[int_cols].astype('int', raise_on_error=False)
        df_price['code'] = code
        df_price.to_sql('stock_price', conn, if_exists='append', index=False)
        print('%d,' % p, end='')
    print()

In [ ]:
stock_code_price_to_sql('105560')

In [ ]:
conn.close()

# 데이터 확인

In [ ]:
%matplotlib inline

import sqlite3
conn = sqlite3.connect('findata.db')

df_price = pd.read_sql('SELECT * FROM stock_price limit 30', conn)
df_price.head()

# 테이블  읽기
df_price = pd.read_sql(
    'SELECT * '
    'FROM stock_price '
    'WHERE code="105560" and date BETWEEN "2015-01-01" AND "2016-04-30"', 
    conn,
    index_col='date')

conn.close()

df_price['close'].plot(figsize=(14,4))

# 전체 다운로드
(주의: 전체 다운로드를 위해서는 4~5일 정도 소요됩니다)

In [ ]:
# 마스터(코드+종목명) 읽기
conn = sqlite3.connect('findata.db')
df_master = pd.read_sql('SELECT * FROM stock_master', conn)
df_master.head()

# 처음 2개 종목만,
for inx, row in df_master[:2].iterrows(): 
    stock_code_price_to_sql(row['code'])

# 모든종목에 대해 실행 하려면, 아래 코드 실행
# for inx, row in df_master.iterrows(): 
#     stock_code_price_to_sql(row['code'])

# 코드 전체 (stock_price)
* stock_price 테이블을 업데이트하는 최종 코드 입니다.
* stock_master 테이블이 미리 생성되어 있어야 합니다.
* 다음 코드를 .py 파일로 저장한 뒤 실행하면 됩니다. 

In [ ]:
%%file stock_price.py
#!/usr/bin/python

# stock_price.py
# findata.db 

import sqlite3
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import requests

def get_last_page_num(code):
    npage = 1
    url = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=1' % (code)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    td = soup.find('td', attrs={'class':'pgRR'})
    if td:
        npage = td.a['href'].split('page=')[1]
    return int(npage)

def get_data_naver(code, start=datetime(1900,1,1), end=datetime(2100,1,1)):
    url_tmpl = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=%d'
    npages = get_last_page_num(code)
    df_price = pd.DataFrame()
    for p in range(1, npages+1):
        url = url_tmpl % (code, p)
        dfs = pd.read_html(url)
        
        # first page
        df = dfs[0] 
        df.columns = ['date', 'close', 'change', 'open', 'high', 'low', 'volume']
        df = df[1:]
        df.dropna(inplace=True)
        df = df.replace('\.', '-', regex=True)

        # select date range
        start_str = start.strftime("%Y-%m-%d")
        end_str = end.strftime("%Y-%m-%d")
        mask = (df['date'] >= start_str) & (df['date'] <= end_str)
        df_in = df[mask]

        # merge dataframe
        df_price = df_price.append(df_in)
        print('%d,' % p, end='', flush=True)
        print(df['date'].max())
        if df['date'].max() <= start_str:
            break
    print()
    df_price['date'] = pd.to_datetime(df_price['date'])
    int_cols = ['close', 'change', 'open', 'high', 'low', 'volume']
    df_price[int_cols] = df_price[int_cols].astype('int', raise_on_error=False)
    df_price.set_index('date', inplace=True)
    return df_price
    
if __name__ == "__main__":
    conn = sqlite3.connect('findata.db')
    df_master = pd.read_sql("SELECT * FROM stock_master", conn)
    for inx, row in df_master.iterrows():
        print(row['code'], row['name'])
        #  start: DB에 저장된 마지막 날짜 + 1일
        df_max = pd.read_sql('SELECT MAX (date) AS "maxdate" FROM stock_price WHERE code="%s"' % row['code'], conn)
        last_date = datetime(1900,1,1)
        if df_max['maxdate'].iloc[0] != None:
            last_date = datetime.strptime(df_max['maxdate'].iloc[0], "%Y-%m-%d %H:%M:%S")
        start = last_date + timedelta(1)

        # end: 전일
        yday = datetime.today() - timedelta(1)
        end = datetime(yday.year, yday.month, yday.day)
        
        df_price = get_data_naver(row['code'], start, end)
        df_price['code'] = row['code']
        df_price.to_sql('stock_price', conn, if_exists='append', index=True)
        print('%d rows' % len(df_price))
    conn.close()

# 실행하기

명령어 라인에서 아래와 같이 실행합니다.

```bash
$ python3 stock_price.py
```

# 인덱스
* 주요한 컬럼(특히, WHERE 절에 들어가는 컬럼)에 인덱스 생성한다.
* 인덱스 여부에 따라 검색 시간이 줄어든다. (예, date의 like검색의 경우 5.1초 --> 6.4 ms)

```sql
$ sqlite3 findata.db                                                                   
SQLite version 3.8.2 2013-12-06 14:53:30
Enter ".help" for instructions
Enter SQL statements terminated with a ";"
sqlite> create index ix_date on stock_price(date);
sqlite> create index ix_date on stock_price(code);
```

----
#### 2017 FinanceData http://fb.com/financedata http://financedata.github.com